In [1]:
from pathlib import Path

import networkx as nx

from python import aml, orc, pes, rdk, sql, hyq

data_dir = Path.cwd() / "data"

connection = sql.connect(data_dir)
sql.initialize_database(connection)


In [2]:
reactants = rdk.mol_from_smiles("[CH2]C")

# goat_temp = orc.Templates.GOAT.XTB
# goat_pars = orc.ORCA_Parameters(8, 1000, 2, "goat.inp", "guess.xyz")

# opt_temp = orc.Templates.Optimization.WB97X_3C
# opt_pars = orc.ORCA_Parameters(8, 1000, 2, "opt.inp", "goat.globalminimum.xyz")

# freq_temp = orc.Templates.Optimization_Frequency.M062x_def2_TZVPP
# freq_pars = orc.ORCA_Parameters(8, 1000, 2, "freq.inp", "opt.xyz")

# calc_temp = orc.Templates.Single_Point_Calculation.CCSDT_F12RI_cc_pVSZ_F12
# calc_pars = orc.ORCA_Parameters(8, 1000, 2, "calc.inp", "freq.xyz")

# stationary_templates = [goat_temp, opt_temp, freq_temp, calc_temp]
# stationary_parameters = [goat_pars, opt_pars, freq_pars, calc_pars]

goat_pars = orc.ORCA_Parameters(
    xyz_in="guess.xyz",
    name_out="goat",
    multiplicity=2,
    functional="XTB",
    method_keywords="GOAT",
)

task_graph = nx.DiGraph()
for reaction_smarts in [
    rdk.Reaction_Templates.PROTON_TRANSFER,
    rdk.Reaction_Templates.RING_OPENING,
]:
    product_sets = rdk.run_reaction(reactants, reaction_smarts, isomorphs=False)
    enumerated_graph = aml.process_rdkit_reaction(reactants, product_sets)
    to_submit = sql.enumerated_graph_into_database(enumerated_graph, data_dir, connection)

    for amchi, kind in to_submit.items():
        if kind == "stationary":
            goat_path = orc.orca_inputs(amchi, goat_pars, data_dir)
            task_graph.add_node(goat_path, pars=goat_pars)

        elif kind == "transition":
            print(amchi)
            continue

connection.close()
hyq.submit_tasks_orca(task_graph)

AMChI=1/C2H5/c1-2-3-1/h1-2H2/k3-2/f3-1


2026-01-07T21:36:12Z INFO A trial allocation was submitted successfully. It was immediately canceled to avoid wasting resources.
2026-01-07T21:36:12Z INFO Allocation queue 7 successfully created
100%|██████████| 1/1 [00:30<00:00, 30.08s/task, 1/1 job completed]


In [ ]:
import automol

enumerated_graph = automol.smiles.graph("[CH2]C[C@@H]1[C@H](C)O1")
geom = automol.graph.geometry(enumerated_graph)
automol.geom.display(geom, label=True)


# for stereo in automol.graph.expand_stereo(graph):
#     geom = automol.graph.geometry(stereo)
#     print(automol.graph.smiles(stereo))
#     automol.geom.display(geom, label=True)

In [ ]:
goat_pars = orc.ORCA_Parameters(8, 1000, 2, "test.inp", "test.xyz")
goat_temp = orc.Templates.GOAT.XTB

pes.orca_inputs(
    "AMChI=1/C5H9O/c1-3-5-4(2)6-5/h4-5H,1,3H2,2H3/t4-,5+/m0/s1",
    goat_temp,
    goat_pars,
    data_dir,
)
